In [2]:
import tensorflow as tf
import numpy as np
import math
import os
import random
import pandas as pd
from pandas import DataFrame
try:
    import cPickle as pickle
except ImportError:
    import _pickle as pickle
import copy

In [3]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [4]:
df_train = pd.read_csv('../data/rl_train_set_unscaled.csv')
df_val = pd.read_csv('../data/rl_val_set_unscaled.csv')
df_test = pd.read_csv('../data/rl_test_set_unscaled.csv')

In [5]:
# let the mortality labels  be -1 and 1: -1 for survival
df_train['died_in_hosp'][df_train['died_in_hosp'] == 0] = -1
df_val['died_in_hosp'][df_val['died_in_hosp'] == 0] = -1
df_test['died_in_hosp'][df_test['died_in_hosp'] == 0] = -1

/afs/csail.mit.edu/u/a/araghu/.conda/envs/my_root/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/afs/csail.mit.edu/u/a/araghu/.conda/envs/my_root/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/afs/csail.mit.edu/u/a/araghu/.conda/envs/my_root/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexi

In [11]:
target_feat = list(np.loadtxt('../data/state_features_pred.txt', dtype=str))
target_feat.append('died_in_hosp')

cur_feat = list(np.loadtxt('../data/state_features.txt', dtype=str))
cur_feat.append('iv_input_norm')
cur_feat.append('vaso_input_norm')

In [7]:
# define an action mapping - how to get an id representing the action from the (iv,vaso) tuple
action_map = {}
count = 0
for iv in range(5):
    for vaso in range(5):
        action_map[(iv,vaso)] = count
        count += 1

In [8]:
inv_action_map = {}
for iv in range(5):
    for vaso in range(5):
        inv_action_map[5*iv+vaso] = [iv,vaso]    

In [9]:
iv_mean = df_train['iv_input'].mean()
iv_std = df_train['iv_input'].std()

vaso_mean = df_train['vaso_input'].mean()
vaso_std = df_train['vaso_input'].std()

In [40]:
df_train['iv_input_norm'] = (df_train['iv_input']- iv_mean)/iv_std
df_train['vaso_input_norm'] = (df_train['vaso_input'] - vaso_mean)/vaso_std

df_val['iv_input_norm'] = (df_val['iv_input']- iv_mean)/iv_std
df_val['vaso_input_norm'] = (df_val['vaso_input'] - vaso_mean)/vaso_std

df_test['iv_input_norm'] = (df_test['iv_input']- iv_mean)/iv_std
df_test['vaso_input_norm'] = (df_test['vaso_input'] - vaso_mean)/vaso_std

In [16]:
#  X: (s_t-3, a_t-3, s_t-2, a_t-2,s_t-1, a_t-1, s_t, a_t )
#  Y: (difference between next state and current state (zeros if end of trajectory), mortality)
hist = 3
def make_data_history_zeros(df_in):
    df_in = df_in.reset_index()
    X = []
    Y = []
    count_in_traj = 0
    for count,i in enumerate(df_in.index):
        if count % 10000 == 0 and count > 0:
            print (count)
        
        # skip the last one; no next state
        if i == df_in.index[-1]:
            break
       
        # if not terminal step in trajectory    
        if df_in.loc[i, 'icustayid'] == df_in.loc[i+1, 'icustayid']:
            count_in_traj += 1
            target = df_in.loc[i + 1, target_feat] - df_in.loc[i, target_feat]
            target[-1] = df_in.loc[i, 'died_in_hosp']
            Y.append(target)
            if count_in_traj >=(hist+1):
                state = df_in.loc[i-hist, cur_feat]
                for index in range(hist-1,-1,-1):
                    state = np.hstack([state,df_in.loc[i-index, cur_feat]])
                X.append(state)
            else:
                num_zeros = (hist+1) - count_in_traj
                num_actual = count_in_traj - 1
                state = np.hstack([np.zeros(len(cur_feat)) for _ in range(num_zeros)])
                for index in range(num_actual, 0, -1):
                    state = np.hstack([state,df_in.loc[i-index, cur_feat]])
                state = np.hstack([state, df_in.loc[i, cur_feat]])
                X.append(state) 
        else:
            count_in_traj = 0

    return np.array(X),np.array(Y)

In [19]:
#  X: (s_t-3, a_t-3, s_t-2, a_t-2,s_t-1, a_t-1, s_t, a_t )
#  Y: (difference between next state and current state (zeros if end of trajectory), mortality)
hist = 3
def make_data_history(df_in):
    df_in = df_in.reset_index()
    X = []
    Y = []
    count_in_traj = 0
    for count,i in enumerate(df_in.index):
        if count % 10000 == 0 and count > 0:
            print (count)
        
        # skip the last one; no next state
        if i == df_in.index[-1]:
            break
       
        # if not terminal step in trajectory    
        if df_in.loc[i, 'icustayid'] == df_in.loc[i+1, 'icustayid']:
            count_in_traj += 1
            if count_in_traj >=(hist+1):

                target = df_in.loc[i + 1, target_feat] - df_in.loc[i, target_feat]
                target[-1] = df_in.loc[i, 'died_in_hosp']
                Y.append(target)
                state = df_in.loc[i-hist, cur_feat]
                for index in range(hist-1,-1,-1):
                    state = np.hstack([state,df_in.loc[i-index, cur_feat]])
                #state = np.hstack([df_in.loc[i-2, cur_feat], df_in.loc[i-1, cur_feat], df_in.loc[i, cur_feat]])
                X.append(state)
            else:
                continue
        else:
            count_in_traj = 0

    return np.array(X),np.array(Y)

In [20]:
#  X: (states, actions)
#  Y: (difference between next state and current state (zeros if end of trajectory), mortality)
def make_data(df_in):
    X = []
    Y = []
    for count,i in enumerate(df_in.index):
        cur_state = df_in.loc[i,cur_feat]
        if i != df_in.index[-1]:
            # if not terminal step in trajectory             
            if df_in.loc[i, 'icustayid'] == df_in.loc[i+1, 'icustayid']:
                target = df_in.loc[i + 1, target_feat] - df_in.loc[i, target_feat]
                target[-1] = df_in.loc[i, 'died_in_hosp']
                Y.append(target)
                X.append(cur_state)
        if count % 10000 == 0 and count > 0:
            print (count)

    return np.array(X),np.array(Y)

In [21]:
# dire = 'converted_data/'
# if not os.path.exists(dire):
#     os.makedirs(dire)
    
# if not os.path.exists(dire + 'X_train_hist.txt'):
#     x_train_hist, y_train_hist = make_data_history(df_train)
#     np.savetxt(dire + 'X_train_hist.txt',x_train_hist,fmt='%5.4f')
#     np.savetxt(dire + 'Y_train_hist.txt',y_train_hist,fmt='%5.4f')
#     print "Saved train"
# else:
#     x_train_hist = np.loadtxt(dire + 'X_train_hist.txt')
#     y_train_hist = np.loadtxt(dire + 'Y_train_hist.txt')
#     print "Loaded train"

# if not os.path.exists(dire + 'X_val_hist.txt'):
#     x_val_hist,y_val_hist = make_data_history(df_val)
#     np.savetxt(dire + 'X_val_hist.txt',x_val_hist,fmt='%5.4f')
#     np.savetxt(dire + 'Y_val_hist.txt',y_val_hist,fmt='%5.4f')
#     print "Saved val"
# else:
#     x_val_hist = np.loadtxt(dire + 'X_val_hist.txt')
#     y_val_hist = np.loadtxt(dire + 'Y_val_hist.txt')
#     print "Loaded val"
    
# if not os.path.exists(dire + 'X_test_hist.txt'):
#     x_test_hist, y_test_hist = make_data_history(df_test)
#     np.savetxt(dire + 'X_test_hist.txt',x_test_hist,fmt='%5.4f')
#     np.savetxt(dire + 'Y_test_hist.txt',y_test_hist,fmt='%5.4f')
#     print "Saved test"
# else:
#     x_test_hist = np.loadtxt(dire + 'X_test_hist.txt')
#     y_test_hist = np.loadtxt(dire + 'Y_test_hist.txt')
#     print "Loaded test"

In [22]:
# dire = 'converted_data/'
# if not os.path.exists(dire):
#     os.makedirs(dire)
    
# if not os.path.exists(dire + 'X_train.txt'):
#     x_train_nohist, y_train_nohist = make_data(df_train)
#     np.savetxt(dire + 'X_train.txt',x_train_nohist,fmt='%5.4f')
#     np.savetxt(dire + 'Y_train.txt',y_train_nohist,fmt='%5.4f')
#     print "Saved train"
# else:
#     x_train_nohist = np.loadtxt(dire + 'X_train.txt')
#     y_train_nohist = np.loadtxt(dire + 'Y_train.txt')
#     print "Loaded train"

# if not os.path.exists(dire + 'X_val.txt'):
#     x_val_nohist,y_val_nohist = make_data(df_val)
#     np.savetxt(dire + 'X_val.txt',x_val_nohist,fmt='%5.4f')
#     np.savetxt(dire + 'Y_val.txt',y_val_nohist,fmt='%5.4f')
#     print "Saved val"
# else:
#     x_val_nohist = np.loadtxt(dire + 'X_val.txt')
#     y_val_nohist = np.loadtxt(dire + 'Y_val.txt')
#     print "Loaded val"
    
# if not os.path.exists(dire + 'X_test.txt'):
#     x_test_nohist, y_test_nohist = make_data(df_test)
#     np.savetxt(dire + 'X_test.txt',x_test_nohist,fmt='%5.4f')
#     np.savetxt(dire +    'Y_test.txt',y_test_nohist,fmt='%5.4f')
#     print "Saved test"
# else:
#     x_test_nohist = np.loadtxt(dire + 'X_test.txt')
#     y_test_nohist = np.loadtxt(dire + 'Y_test.txt')
#     print "Loaded test"

In [24]:
dire = 'converted_data/'
if not os.path.exists(dire):
    os.makedirs(dire)

if not os.path.exists(dire + 'X_train_hist_zeros.txt'):
    train_feat_zeros, train_labels_zeros = make_data_history_zeros(df_train)
    np.savetxt(dire + 'X_train_hist_zeros.txt',train_feat_zeros,fmt='%5.4f')
    np.savetxt(dire + 'Y_train_hist_zeros.txt',train_labels_zeros,fmt='%5.4f')
    print ("Saved train_zeros")
else:
    train_feat_zeros = np.loadtxt(dire + 'X_train_hist_zeros.txt')
    train_labels_zeros = np.loadtxt(dire + 'Y_train_hist_zeros.txt')
    print ("Loaded train_zeros")

if not os.path.exists(dire + 'X_val_hist_zeros.txt'):
    val_feat_zeros, val_labels_zeros = make_data_history_zeros(df_val)
    np.savetxt(dire + 'X_val_hist_zeros.txt',val_feat_zeros,fmt='%5.4f')
    np.savetxt(dire + 'Y_val_hist_zeros.txt',val_labels_zeros,fmt='%5.4f')
    print ("Saved val_zeros")
else:
    val_feat_zeros = np.loadtxt(dire + 'X_val_hist_zeros.txt')
    val_labels_zeros = np.loadtxt(dire + 'Y_val_hist_zeros.txt')
    print ("Loaded val_zeros")

if not os.path.exists(dire + 'X_test_hist_zeros.txt'):
    test_feat_zeros, test_labels_zeros = make_data_history_zeros(df_test)
    np.savetxt(dire + 'X_test_hist_zeros.txt',test_feat_zeros,fmt='%5.4f')
    np.savetxt(dire + 'Y_test_hist_zeros.txt',test_labels_zeros,fmt='%5.4f')
    print ("Saved test_zeros")
else:
    test_feat_zeros = np.loadtxt(dire + 'X_test_hist_zeros.txt')
    test_labels_zeros = np.loadtxt(dire + 'Y_test_hist_zeros.txt')
    print ("Loaded test_zeros")

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
Saved train_zeros
10000
20000
Saved val_zeros
10000
20000
30000
40000
Saved test_zeros


In [26]:
# Add Gaussian noise to the state features. Commented out for now -- might need to put back in.
# gaussian_shape = df.loc[:, state_features].values.shape
# noise = np.random.normal(0, 0.03, gaussian_shape)
# df.loc[:, state_features] += noise

In [27]:
x_train = train_feat_zeros
y_train = train_labels_zeros

x_val = val_feat_zeros
y_val = val_labels_zeros

x_test = test_feat_zeros
y_test = test_labels_zeros

In [28]:
print (x_train.shape, y_train.shape)
print (x_val.shape, y_val.shape)
print (x_test.shape, y_test.shape)

(156967, 200) (156967, 43)
(22549, 200) (22549, 43)
(45036, 200) (45036, 43)


In [34]:
hidden_1_size = 128
hidden_2_size = 32
class EnvModel():
    def __init__(self):
        self.phase = tf.placeholder(tf.bool)
        
        self.input_size = x_train.shape[1]
        
#         self.target_size = 1 # for now, just predict sofa

        self.target_size = y_train.shape[1]

        self.cur_state = tf.placeholder(tf.float32, shape=[None, self.input_size],name="cur_state")
        self.targets = tf.placeholder(tf.float32, shape=[None, self.target_size],name="target")
        
        self.fc_1 = tf.contrib.layers.fully_connected(self.cur_state, hidden_1_size, activation_fn=tf.nn.relu)
        self.fc_1_bn = tf.contrib.layers.batch_norm(self.fc_1, center=True, scale=True, is_training=self.phase)
        self.fc_2 = tf.contrib.layers.fully_connected(self.fc_1_bn, hidden_2_size, activation_fn=tf.nn.relu)
        self.fc_2_bn = tf.contrib.layers.batch_norm(self.fc_2, center=True, scale=True, is_training=self.phase)
        
        self.output = tf.contrib.layers.fully_connected(self.fc_2_bn, self.target_size, activation_fn = None)
        
        self.loss = tf.reduce_mean(tf.square(self.targets-self.output))
        
        #self.est_next_state = self.output + self.cur_state
        
        self.reg_lambda = 0.0000
        
        self.vars = tf.trainable_variables() 
        self.lossL2 = tf.add_n([ tf.nn.l2_loss(v) for v in self.vars
                    if 'bias' not in v.name ]) * self.reg_lambda    
        
        self.trainer = tf.train.AdamOptimizer(learning_rate=0.001)
        self.update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(self.update_ops):
        # Ensures that we execute the update_ops before performing the model update, so batchnorm works
            self.update_model = self.trainer.minimize(self.loss + self.lossL2)

In [35]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # Don't use all GPUs 
config.allow_soft_placement = True  # Enable manual control

In [36]:
eval_batch = 64
def run_eval(sess, env_mdl):
    start = 0
    end = 0
    net_loss = 0.0
    while start < len(x_val):
        end = min(len(x_val), start+eval_batch)
        x_batch = x_val[start:end]
        y_batch = y_val[start:end]

        # extract out only sofa for target
#         y_batch = y_batch[:, 29]
#         y_batch = y_batch[:, None]

        # train using the batch
        loss = sess.run(env_model.loss,
                        feed_dict={env_model.cur_state:x_batch,
                                   env_model.targets:y_batch,
                                   env_model.phase:False})
        # update net loss
        net_loss += loss

        # increment start index to inds
        start += eval_batch
    
    avg_eval_loss = net_loss/(len(x_val)/float(eval_batch))
    
    return avg_eval_loss
        

In [39]:
batch_size = 64
num_epochs = 30
load_model = False #Whether to load a saved model.
save_dir = "./env_model_reg/"
save_path = "./env_model_reg/ckpt"#The path to save our model to.
tf.reset_default_graph()
env_model = EnvModel()

saver = tf.train.Saver(tf.global_variables())

init = tf.global_variables_initializer()

trainables = tf.trainable_variables()

#Make a path for our model to be saved in.
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

with tf.Session(config=config) as sess:
    if load_model == True:
        print('Trying to load model...')
        try:
            restorer = tf.train.import_meta_graph(save_path + '.meta')
            restorer.restore(sess, tf.train.latest_checkpoint(save_dir))
            print ("Model restored")
        except IOError:
            print ("No previous model found, running default init")
            sess.run(init)
    else:
        print("Running default init")
        sess.run(init)
    print("Init done")
    
    for j in range(1,num_epochs+1):
        net_loss = 0.0
        inds = np.random.permutation(x_train.shape[0])
        start_idx = 0
        end_idx = 0
        while start_idx < len(x_train):
            end_idx = min(len(x_train), start_idx+batch_size)
            batch_inds = inds[start_idx:end_idx]
            x_batch = x_train[batch_inds]
            y_batch = y_train[batch_inds]
            
            # extract out only sofa for target
#             y_batch = y_batch[:, 29]
#             y_batch = y_batch[:, None]
            
            # train using the batch
            _,loss = sess.run([env_model.update_model,env_model.loss], \
            feed_dict={env_model.cur_state:x_batch,
                       env_model.targets:y_batch,
                       env_model.phase:True})
            
            # update net loss
            net_loss += loss
            
            # increment start index to inds
            start_idx += batch_size
        
        saver.save(sess,save_path)
        print("Saved Model, epoch " + str(j))
        
        av_loss = net_loss/(len(x_train)/float(batch_size))
        print("Average training loss per batch is ", av_loss)
        net_loss = 0.0
        
        eval_loss = run_eval(sess, env_model)
        
        print ("Avg val loss at epoch end is " + str(eval_loss))

    

Running default init
Init done
Saved Model, epoch 1
Average training loss per batch is  0.184884197271
Avg val loss at epoch end is 0.15933036045
Saved Model, epoch 2
Average training loss per batch is  0.159051720528
Avg val loss at epoch end is 0.153515678239
Saved Model, epoch 3
Average training loss per batch is  0.155490289914
Avg val loss at epoch end is 0.151085885783
Saved Model, epoch 4
Average training loss per batch is  0.153782491525
Avg val loss at epoch end is 0.150342578068
Saved Model, epoch 5
Average training loss per batch is  0.152663462748
Avg val loss at epoch end is 0.14904760643
Saved Model, epoch 6
Average training loss per batch is  0.151809316994
Avg val loss at epoch end is 0.148571506277
Saved Model, epoch 7
Average training loss per batch is  0.15113408074
Avg val loss at epoch end is 0.148279145677
Saved Model, epoch 8
Average training loss per batch is  0.150748146016
Avg val loss at epoch end is 0.14781003284
Saved Model, epoch 9
Average training loss pe

In [62]:
with tf.Session(config=config) as sess:
    print('Trying to load model...')
    try:
        restorer = tf.train.import_meta_graph(save_path + '.meta')
        restorer.restore(sess, tf.train.latest_checkpoint(save_dir))
        print "Model restored"
    except IOError:
        print "No previous model found, running default init"
        sess.run(init)
    print("Init done")
    
    bx = x_train[50:100]
    by = y_train[50:100, 29]
    by = by[:, None]
    op = sess.run(env_model.output, \
            feed_dict={env_model.cur_state:bx,
                       env_model.targets:by,
                       env_model.phase:False})

# print op.shape, by.shape
# for i in zip(np.squeeze(op), np.squeeze(by)):
#     print i

Trying to load model...
Model restored
Init done


In [63]:
np.mean((np.squeeze(op)-np.squeeze(by))**2)

0.11017323824480833

In [64]:
for i in zip(np.squeeze(op), np.squeeze(by)):
    print i

(0.17911743, 0.0)
(0.093046457, 0.0)
(0.13156834, 0.28613931514300006)
(0.075183168, 0.0)
(0.15709624, 1.144557260572)
(0.053191449, 0.28613931513999979)
(0.007503923, -0.28613931513999979)
(0.086839095, 0.0)
(0.059295628, 0.0)
(-0.0029163994, 0.0)
(0.02531508, 0.28613931513999979)
(-0.036283854, 0.0)
(-0.039782193, 0.0)
(-0.10121112, -0.57227863027999981)
(-0.16417766, -0.28613931514799995)
(0.13331677, 0.0)
(0.053894948, 0.0)
(0.21118657, 0.28613931514799995)
(0.16026877, 0.0)
(0.082047328, 0.28613931514000002)
(-0.057933476, 0.0)
(0.035227526, 0.28613931513999979)
(-0.036803689, 0.0)
(-0.12285101, -0.28613931513999979)
(-0.14585981, 0.0)
(-0.14774466, -0.28613931514000002)
(-0.19879135, -0.28613931514799995)
(-0.043731753, 0.0)
(0.027128305, 0.0)
(-0.092596322, -1.144557260572)
(0.24658708, 0.0)
(0.16986302, 0.0)
(0.16923501, 0.0)
(0.14478707, 0.0)
(0.09138447, 0.0)
(0.12362672, 0.0)
(0.057311695, 0.0)
(0.064386889, 0.5722786302859999)
(-0.077138633, 1.144557260574)
(-0.51417947, -0